In [ ]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt

In [ ]:
import time
import datetime
import numpy as np
from pathlib import Path

In [ ]:
from ophyd import Device, Component as Cpt, Signal
from ophyd.sim import NullStatus, new_uid

In [ ]:
from ophyd.utils import make_dir_tree
_ = make_dir_tree(datetime.datetime.now().year, base_path='/tmp/data')

In [ ]:
class LaptopCamFileHandler:
    specs = {'laptop_cam'}

    def __init__(self, filename):
        self._name = filename

    def __call__(self):
        return np.load(self._name)

In [ ]:
from databroker import Broker, temp_config
db = Broker.from_config(temp_config())
db.reg.register_handler('laptop_cam', LaptopCamFileHandler, overwrite=True)

In [ ]:
from bluesky.run_engine import RunEngine
RE = RunEngine({})
token = RE.subscribe(db.insert)

In [ ]:
import bluesky.plans as bp

In [ ]:
class LaptopCam(Device):
    image = Cpt(Signal)
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._resource_id = None
        self._npy_file = None
        self.det = cv2.VideoCapture(1)
        
    def trigger(self):
        super().trigger()

        datum_id = new_uid()
        date = datetime.datetime.now()
        self._npy_file = Path('/tmp/data') / Path(date.strftime('%Y/%m/%d')) / \
            Path('{}.npy'.format(datum_id))
        print(f'npy_file: {self._npy_file}')
        
        self.det = cv2.VideoCapture(0)
        _, frame = self.det.read()
        print(frame)

        np.save(self._npy_file, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        self.image.put(datum_id)
        self._resource_id = db.reg.insert_resource('laptop_cam', self._npy_file, {})
        db.reg.insert_datum(self._resource_id, datum_id, {})
        self.det.release()
        time.sleep(1)
        return NullStatus()
    
    def describe(self):
        res = super().describe()
        res[self.image.name].update(dict(external="FILESTORE"))
        return res
    
    def unstage(self):
        super().unstage()
        self.det.release()

In [ ]:
lc = LaptopCam(name='lc')

In [ ]:
RE(bp.count([lc], num=3))

In [ ]:
hdr = db[-1]

In [ ]:
hdr.table()

In [ ]:
hdr.fields()

In [ ]:
imgs = np.stack(hdr.data('lc_image'))

In [ ]:
imgs.shape

In [ ]:
plt.imshow(imgs[0])

In [ ]:
plt.imshow(imgs[1])

In [ ]:
plt.imshow(imgs[2])